In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
from tqdm import tqdm
import features_extraction_modul as fem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

In [4]:
ef_path = "extracted_features_large/"
files = os.listdir(ef_path)

In [5]:
x_pd_array = []
y_pd_array = []
for f in files:
    print(f)
    i_ef =  pd.read_csv(ef_path+f)
    if f[0] == "x":
        x_pd_array.append(i_ef)
    else:
        y_pd_array.append(i_ef)

x_train00.csv
x_train01.csv
x_train02.csv
x_train03.csv
x_train04.csv
x_train05.csv
x_train06.csv
x_train07.csv
x_train08.csv
x_train09.csv
x_train10.csv
x_train11.csv
x_train12.csv
x_train13.csv
x_train14.csv
x_train_t9_02.csv
x_train_t9_07.csv
x_train_t9_14.csv
y_train00.csv
y_train01.csv
y_train02.csv
y_train03.csv
y_train04.csv
y_train05.csv
y_train06.csv
y_train07.csv
y_train08.csv
y_train09.csv
y_train10.csv
y_train11.csv
y_train12.csv
y_train13.csv
y_train14.csv
y_train_t9_02.csv
y_train_t9_07.csv
y_train_t9_14.csv


In [6]:
x_train = pd.concat(x_pd_array, ignore_index = True).iloc[:, 1:]
y_train = pd.concat(y_pd_array, ignore_index = True).iloc[:, 1:]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 101)

In [9]:
#normalizing the data
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

y_train_flatten = y_train.values.ravel() #flattening the y_train
y_test_flatten = y_test.values.ravel() #flattening the y_test

In [10]:
print(x_train_scaled.shape)
print(x_test_scaled.shape)
print(y_train_flatten.shape)
print(y_test_flatten.shape)

(4220, 98)
(1056, 98)
(4220,)
(1056,)


In [140]:
def GetModel():
    nInputs = x_train_scaled.shape[1]
    inputs = tf.keras.Input(shape=(nInputs, ), name = "features")
    activation = tf.keras.layers.Activation(tf.nn.leaky_relu)
    x = inputs
    for iLayer in range(0,6):
        dense = 1024/2**(iLayer)
        x = tf.keras.layers.Dense(dense, activation=activation, name="layer_{}".format(iLayer))(x)
        x = tf.keras.layers.Dropout(0.35)(x)
    
    outputs = tf.keras.layers.Dense(1, activation=tf.keras.activations.relu,
                               bias_initializer='zeros',
                               name = "time")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name = "NN1")
    initial_learning_rate = 0.00005
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.95,
    staircase=True)
  
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                loss=tf.keras.losses.MeanAbsoluteError(),
                metrics=['mean_absolute_error'])
    tf.keras.utils.plot_model(model, 'ML_model.png', show_shapes=True)
    return model
    
    

In [141]:
model = GetModel()
model.summary()
nEpochs = 100

#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

history = model.fit(x_train_scaled, y_train_flatten , epochs=nEpochs,
                    use_multiprocessing=True,
                    verbose=2,
                    shuffle=True,
                    validation_data= (x_test_scaled, y_test_flatten),
                    #callbacks=[callback]
                   )


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Model: "NN1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (InputLayer)        [(None, 98)]              0         
_________________________________________________________________
layer_0 (Dense)              (None, 1024)              101376    
_________________________________________________________________
dropout_63 (Dropout)         (None, 1024)              0         
_________________________________________________________________
layer_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_64 (Dropout)         (None, 512)               0         
_________________________________________________________________
layer_2 (Dense)          

132/132 - 1s - loss: 2.6626 - mean_absolute_error: 2.6626 - val_loss: 2.6047 - val_mean_absolute_error: 2.6047
Epoch 49/100
132/132 - 1s - loss: 2.6609 - mean_absolute_error: 2.6609 - val_loss: 2.5709 - val_mean_absolute_error: 2.5709
Epoch 50/100
132/132 - 1s - loss: 2.6690 - mean_absolute_error: 2.6690 - val_loss: 2.6738 - val_mean_absolute_error: 2.6738
Epoch 51/100
132/132 - 1s - loss: 2.6887 - mean_absolute_error: 2.6887 - val_loss: 2.5829 - val_mean_absolute_error: 2.5829
Epoch 52/100
132/132 - 1s - loss: 2.6531 - mean_absolute_error: 2.6531 - val_loss: 2.5757 - val_mean_absolute_error: 2.5757
Epoch 53/100
132/132 - 1s - loss: 2.6369 - mean_absolute_error: 2.6369 - val_loss: 2.5079 - val_mean_absolute_error: 2.5079
Epoch 54/100
132/132 - 1s - loss: 2.6736 - mean_absolute_error: 2.6736 - val_loss: 2.6115 - val_mean_absolute_error: 2.6115
Epoch 55/100
132/132 - 1s - loss: 2.6365 - mean_absolute_error: 2.6365 - val_loss: 2.5781 - val_mean_absolute_error: 2.5781
Epoch 56/100
132/132 

In [131]:
prediction_score = []
mae_scores = []
predictions = model.predict(x_test_scaled, verbose = 1) #make predictons on the test data 
prediction_score.append(predictions)

mae = mean_absolute_error(predictions, y_test_flatten) #calculating MAE between our actual output labels and the predicted otuput labels
mae_scores.append(mae)

33/33 [==============================] - 0s 3ms/step


In [132]:
print(np.mean(mae_scores)) #average of mean absolute errors


2.5654110937400465


In [133]:
ef_sample_submission = pd.read_csv("ef_sample_submission.csv")

In [134]:
ef_sample_submission = ef_sample_submission.iloc[:, 1:]
seg =  ef_sample_submission.iloc[:, 0:1]
ef_sample_submission = ef_sample_submission.iloc[:, 1:]

In [135]:
X_test_scaled = scaler.transform(ef_sample_submission)


In [136]:
predictions = model.predict(X_test_scaled, verbose = 1)


82/82 [==============================] - 0s 4ms/step


In [137]:
predictions

array([[ 3.550687 ],
       [ 7.5158644],
       [ 5.4722347],
       ...,
       [ 3.8041677],
       [ 1.8148482],
       [12.161914 ]], dtype=float32)

In [138]:
np.column_stack([seg, predictions])

array([['seg_00030f', 3.550687074661255],
       ['seg_0012b5', 7.515864372253418],
       ['seg_00184e', 5.472234725952148],
       ...,
       ['seg_ff79d9', 3.8041677474975586],
       ['seg_ffbd6a', 1.8148481845855713],
       ['seg_ffe7cc', 12.161913871765137]], dtype=object)

In [139]:
final = pd.DataFrame(np.column_stack([seg, predictions]))
final["seg_id"] =  final.iloc[:, 0:1]
final["time_to_failure"] = final.iloc[:, 1:2]
final = final.iloc[:, 2:]
final["time_to_failure"] = final["time_to_failure"].astype("float64")
final.to_csv("sample_submission.csv", index = False )